In [156]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.distributions.distribution as dist
import torch.distributions.categorical as Categorical

import math
from functools import reduce
import datetime
from dateutil.relativedelta import relativedelta
from datetime import date
import sys
import os
import pickle
import json
import copy
import time
import operator
import numpy as np
import random
from collections import namedtuple, deque
import matplotlib.pyplot as plt
import imageio

# Solving the Elevator problem

The goal of elevators is to deliver people to their destination in an efficient manner.


## The Goal

Design a system that solves the elevator problem by minimizing the amount of energy required for each elevator, the wait times for all passengers. 

**Two Types of Elevator systems**
1. up/down button outside, exact floor inside
2. input destination outside

Constraints:
- Time taken within the elevator is valued less than time waiting at a floor
- Predictive analytics for where people are going to be. (Elevators sent to the most popular floor if no route)
- Maximum allowable wait period
- Energy used by each elevator
- Elevator capacity

could make each floor an object, that contains people (if any).
could make each person an object that contains current floor and destination
could make each person a tuple that is contained within the elevator class
Separate people into inside the elevator and waiting for elevator. and done.
Doesn't matter if there are multiple people on each floor. Its effectively 1 person, except for when it reaches capacity. 


## The training structure

the training algorithm will go by steps. each step will add the current amount of time spent by people inside and outside the elevators to a collective time spent. That will be the reward signal. 

1. **Rewards**: Collective time spent waiting (can be weighted to bias for time inside or outside the elevator)
2. **Done**: True when all passengers have been delivered
3. **Penalty**: for exceeding maximum wait time. If this is true, then the agent must know the start time for each passenger.
4. **Step**: increments elevators towards their current destinations. increments cumulative time waited.
5. **Elevators**: Only pick people up when they reach a destination floor, and or drop people off.

**Initially solve for the simpliest example of one person and one elevator**

### Input

The remaining people on each floor, their destinations and time they pressed the button. Locations of elevators and their destinations. The notion of people is a little strange as its purely about how many different destinations (if any) per floor

1 hot representation of floors. each floor has a 1 hot representation of the destinations. where 0 or more can be turned on. 
1 hot representation of elevator locations,and elevator destinations, and the destinations of the people inside the elevators.

- N_planes = N_people + N_destinations + N_elevators + N_elevator_targets + N_elevator_destinations
- Input shape (N_planes,N_floors)

### Actions

- consist of destinations for each elevator.
- action shape (N_elevators,N_floors)

In [1087]:
class Hotel(object):
    def __init__(self,N_floors,N_elevators,N_people,capacity=None):
        """
        N_floors : Int, Number of floors for the building
        N_elevators : Int, Number of elevators
        N_people : Int, Number of people
        L_destinations : list, 
        Time_weights : list of weights, [0] weight for outside elevator, [1] weight for inside elevator
        """
        self.N_floors = N_floors
        self.N_elevators = N_elevators
        self.N_people = N_people
        
        # Constraints
        self.capacity = capacity
        self.time_weights = [1,1]
        self.reset()
        
    def step(self,action,printing=True):
        """
        count all time spent
        check if any elevators picked up or dropped people off
        return new state
        """
        self.move_elevators(action)
        self.check_elevators()
        reward = self.count_time()
        state = self.return_state()
        if printing == True:
            print(str(self))
        return state,reward,self.isdone
        
    def count_time(self):
        outside_mask = np.where(self.floors==1)[0]
        num_o_waiting = outside_mask.shape[0]
        num_o_waiting *= self.time_weights[0]
        # Inside elevators
        inside_mask = np.where(self.elevator_destinations == 1)[0]
        num_e_waiting = inside_mask.shape[0]
        num_e_waiting *= self.time_weights[1]
        self.cumulative_time += num_e_waiting + num_o_waiting
        return num_e_waiting + num_o_waiting
    
    def move_elevators(self,action):
        self.elevator_targets = action
        # Move elevators 1 in the direction of the target
        # Get the vertical distance
        loc_mask = np.where(self.elevator_locations==1)[1]
        tar_mask = np.where(self.elevator_targets==1)[1]
        vert_distance = tar_mask - loc_mask
        elevators = np.arange(self.N_elevators)
        move = np.clip(vert_distance,-1,1)
        # print('move elevators',move,loc_mask)
        move_mask = loc_mask + move
        self.elevator_locations[elevators,loc_mask] = 0 
        self.elevator_locations[elevators,move_mask] = 1
        
    def check_elevators(self):
        loc_mask = np.where(self.elevator_locations==1)[1]
        tar_mask = np.where(self.elevator_targets==1)[1]
        vert_distance = tar_mask - loc_mask
        # If any target == the location, drop/pickup
        if 0 in vert_distance:
            # print('at location')
            stopped_mask = np.where(vert_distance == 0)[0]
            # print(self.elevator_destinations[stopped_mask])
            # print(self.elevator_locations[stopped_mask])
            # Check elevator destinations
            if 1 in self.elevator_destinations[stopped_mask]:
                for el in stopped_mask:
                    loc_mask = np.where(self.elevator_locations[el] == 1)
                    dest_mask = np.where(self.elevator_destinations[el] == 1)[0]
                    if loc_mask in dest_mask:
                        # Drop off
                        # print('passenger dropped off')
                        self.elevator_destinations[el,dest_mask] = 0
                        
                    
            # Check pickups
            if 1 in self.floors:
                # print('check pickups')
                for el in stopped_mask:
                    loc_mask = np.where(self.elevator_locations[el] == 1)[0] # elevator floor index
                    which_person = np.where(self.floors[:,loc_mask] == 1)[0] # Which person is at that floor (if any)
                    if which_person.size > 0:
                        # print('passenger picked up')
                        self.floors[which_person,:] = 0
                        self.elevator_destinations[[el]] += self.destinations[which_person]
                        self.destinations[which_person,:] = 0
                
    def return_state(self):
        # Concat together all floor and elevator states
        # print('elevator_locations.shape',self.elevator_locations.shape)
        # print('elevator_targets',self.elevator_targets.shape)
        # print('elevator_destinations',self.elevator_destinations.shape)
        machine_input = np.concatenate([self.floors,self.destinations,self.elevator_locations,self.elevator_targets,self.elevator_destinations],axis=0)
        # print('return state final',machine_input.shape)
        return np.expand_dims(machine_input, axis=0)
    
    def reset(self,top=True):
        # Floors
        self.floors = np.zeros((self.N_people,self.N_floors))
        self.destinations = np.zeros((self.N_people,self.N_floors))
        # Elevators
        self.elevator_locations = np.zeros((self.N_elevators,self.N_floors))
        self.elevator_targets = np.zeros((self.N_elevators,self.N_floors))
        self.elevator_destinations = np.zeros((self.N_elevators,self.N_floors))
        self.cumulative_time = 0
        # Generate floor destinations
        low = self.N_floors-1 if top == True else 0
        high = self.N_floors
        people_locations = np.random.choice(np.arange(self.N_floors),self.N_people,replace=False)
        people_destinations = np.random.randint(low,high,self.N_people)
        people = np.arange(self.N_people)
        self.floors[people,people_locations] = 1
        self.destinations[people,people_destinations] = 1
        # Generate elevator locations
        low_e = 0
        high_e = 1
        elevators = np.arange(self.N_elevators)
        elevator_locations = np.random.randint(low_e,high_e,self.N_elevators)
        self.elevator_locations[elevators,elevator_locations] = 1
        return self.return_state()
            
    @property
    def isdone(self):
        # Check if finish condition is met`
        outside_mask = np.where(self.floors==1)[0]
        num_o_waiting = outside_mask.shape[0]
        inside_mask = np.where(self.elevator_destinations == 1)[0]
        num_e_waiting = inside_mask.shape[0]
        if num_e_waiting + num_o_waiting == 0:
            finished = 1
        else:
            finished = 0
        return finished
    
    @property
    def action_space(self):
        return (self.N_elevators,self.N_floors)
    
    @property
    def state_space(self):
        return np.concatenate([self.floors,self.destinations,self.elevator_locations,self.elevator_targets,self.elevator_destinations],axis=0).shape
    
    # def reset(self):
    #     # Floors
    #     self.floors = np.zeros((self.N_people,self.N_floors))
    #     self.destinations = np.zeros((self.N_people,self.N_floors))
    #     # Elevators
    #     self.elevator_locations = np.zeros((self.N_elevators,self.N_floors))
    #     self.elevator_targets = np.zeros((self.N_elevators,self.N_floors))
    #     self.elevator_destinations = np.zeros((self.N_elevators,self.N_floors))
    #     self.cumulative_time = 0
    
    def __repr__(self):
        return '%s(%r)' % (self.__class__,self.__dict__)
    
    def __str__(self):
        elevator_locations = np.where(hotel.elevator_locations==1)[1]
        elevator_destinations = np.where(hotel.elevator_destinations==1)[1]
        people_locations = np.where(hotel.floors==1)[1]
        destinations = np.where(hotel.destinations==1)[1]
        return "elevator_locations {}, elevator_destinations {}, people_locations {}, destinations {}"\
    .format(elevator_locations,elevator_destinations,people_locations,destinations)
    

# Tests

In [1088]:
N_floors = 5
N_elevators = 1
N_people = 1

hotel = Hotel(N_floors,N_elevators,N_people)

## New scenario

In [1089]:
hotel.reset()
print(repr(hotel))
print(hotel)

<class '__main__.Hotel'>({'N_floors': 5, 'N_elevators': 1, 'N_people': 1, 'capacity': None, 'time_weights': [1, 1], 'floors': array([[0., 0., 0., 0., 1.]]), 'destinations': array([[0., 0., 0., 0., 1.]]), 'elevator_locations': array([[1., 0., 0., 0., 0.]]), 'elevator_targets': array([[0., 0., 0., 0., 0.]]), 'elevator_destinations': array([[0., 0., 0., 0., 0.]]), 'cumulative_time': 0})
elevator_locations [0], elevator_destinations [], people_locations [4], destinations [4]


## Machine state

In [1056]:
hotel.return_state().shape

(1, 5, 5)

## Move elevators

In [1057]:
test_action = np.zeros((N_elevators,N_floors))
test_action[0,4] = 1 # Move up
# test_action[0,0] = 1 # Move down
# 2 elevators
# test_action = np.zeros((N_elevators,N_floors))
# test_action[0,4] = 1 # Move up
# # test_action[1,2] = 1 # Move up
# test_action[0,4] = 1 # Move down
# test_action[1,4] = 1 # Move down
print('action',test_action)
hotel.move_elevators(test_action)

action [[0. 0. 0. 0. 1.]]


In [1058]:
str(hotel)

'elevator_locations [1], elevator_destinations [], people_locations [1], destinations [4]'

## Check elevators

In [1063]:
hotel.check_elevators()

## Step

In [1064]:
hotel.reset()
str(hotel)
hotel.return_state()

array([[[0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 1.],
        [1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]])

In [1065]:
test_action = np.zeros((N_elevators,N_floors))
test_action[0,4] = 1 # Move up
hotel.step(test_action)

TypeError: check_elevators() takes 1 positional argument but 2 were given

In [1092]:
class Config(object):
    def __init__(self,agent): 
        if agent == 'PPO':
            self.name = agent
            self.gae_lambda=0.95
            self.num_agents=1
            self.batch_size=32
            self.gradient_clip=10
            self.SGD_epoch=10
            self.epsilon=0.2
            self.beta=0.01
            self.gamma=0.99
            self.lr = 1e-4
            self.L2 = 0.01
            self.checkpoint_path = 'model_weights/PPO.ckpt'
        elif agent == "ddpg":
            self.seed = 99
            self.name = agent
            self.num_agents = 2
            self.QLR = 0.001
            self.ALR = 0.0001
            self.gamma = 0.99
            self.L2 = 0 # 0.1
            self.tau=0.01 # 0.001
            self.noise_decay=0.99995
            self.gae_lambda = 0.97
            self.clip_norm = 10
            # Buffer
            self.buffer_size = int(1e4)
            self.min_buffer_size = int(1e3)
            self.batch_size = 256
            # Priority Replay
            self.ALPHA = 0.6 # 0.7 or 0.6
            self.START_BETA = 0.5 # from 0.5-1
            self.END_BETA = 1
            # distributional
            self.N_atoms = 51
            self.v_min = -2
            self.v_max = 2
            self.delta_z = (self.v_min - self.v_max) / (self.N_atoms - 1)
            # Tennis
            self.action_low=-1.0 
            self.action_high=1.0
            self.winning_condition = 10
            # Training
            self.episodes = 100
            self.tmax = 2000
            self.print_every = 4
            self.SGD_epoch = 1
            self.checkpoint_path = 'model_weights/ddpg.ckpt'
        else:
            raise ValueError('Agent not implemented')

In [1076]:
"""
PyTorch implementation of Actor Critic class for PPO. 
Combined torso with dual output 

2 Options:
use categorical for each slice
use softmax and torch.log for whole

Inputs:
Active routes
Historical routes (for novelty)
Current distance (minus stripped routes)
Can use the mask in the foward pass to auto restrict which techniques to suggest.
"""

class PPO_net(nn.Module):
    def __init__(self,nS,nA,seed,indicies,hidden_dims=(256,128)):
        super(PPO_net,self).__init__()
        self.nS = nS
        self.nA = nA
        self.seed = torch.manual_seed(seed)
        self.indicies = indicies
        self.hidden_dims = hidden_dims
        # TODO implement own batchnorm function
        # self.batch = manual_batchnorm()
        
        # Layers
        self.input_layer = nn.Linear(self.nS,hidden_dims[0])
        # self.input_bn = nn.BatchNorm1d(hidden_dims[0])
        self.hidden_layers = nn.ModuleList()
        # self.hidden_batches = nn.ModuleList()
        for i in range(1,len(hidden_dims)):
            # hidden_batch = nn.BatchNorm1d(hidden_dims[i-1])
            hidden_layer = nn.Linear(hidden_dims[i-1],hidden_dims[i])
            self.hidden_layers.append(hidden_layer)
            # self.hidden_batches.append(hidden_batch)

        
        # Action outputs, we softmax over the various classes for 1 per class (can change this for multi class)
        self.actor_output = nn.Linear(hidden_dims[-1],nA)
        self.critic_output = nn.Linear(hidden_dims[-1],1)

    def forward(self,state):
        """
        Expects state to be a torch tensor

        Outputs Action,log_prob, entropy and (state,action) value
        """
        assert isinstance(state,torch.Tensor)
        x = F.relu(self.input_layer(state))
        for i,hidden_layer in enumerate(self.hidden_layers):
            x = F.relu(hidden_layer(x))
            
        # state -> action
        action = self.actor_output(x)
        a_prob = F.softmax(action,dim=0)
        log_prob = torch.log(action)
        # Critic state value
        v = self.critic_output(x)
        return a_prob, log_prob, v

In [1077]:
class PPO(object):
    def __init__(self,nS,nA,config):
        self.nS = nS
        self.nA = nA
        self.seed = config.seed
        self.lr = config.lr

        self.gradient_clip = config.gradient_clip
        self.gamma = config.gamma
        self.gae_lambda = config.gae_lambda
        self.start_epsilon = self.epsilon = config.epsilon
        self.start_beta = self.beta = config.beta
        self.SGD_epoch = config.SGD_epoch
        self.batch_size = config.batch_size

        self.device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

        self.policy = PPO_net(nS,nA,self.seed).to(self.device)
        self.optimizer = optim.Adam(self.policy.parameters(),lr=1e-4,weight_decay=config.L2)

    def load_weights(self,path):
        self.policy.load_state_dict(torch.load(path))
        self.policy.eval()

    def save_weights(self,path):
        directory = os.path.dirname(path)
        if not os.path.exists(directory):
            os.mkdir(directory)
        torch.save(self.policy.state_dict(), path)

    def reset_hyperparams(self):
        self.discount = self.start_discount
        self.epsilon = self.start_epsilon
        self.beta = self.start_beta

    def step_hyperparams(self):
        self.epsilon *= 0.999
        self.beta *= 0.995

    def step(self,trajectory):
        N = len(trajectory)
        # decay beta,epsilon
        self.step_hyperparams()
        
        states,actions,values,log_probs,rewards,next_states = self.unwrap(trajectory)
        # Normalize the rewards
        # rewards = (rewards - np.mean(rewards)) / np.std(rewards)
        last_value = self.policy(self.tensor(next_states[-1]))[-1].unsqueeze(1).cpu().detach().numpy()
        values = np.vstack(values + [last_value])
        advs,TD_errors = self.gae(values,rewards)
        returns = self.n_step_returns(rewards)
        
        states,actions,log_probs,returns,advs,TD_errors,rewards = self.bulk_tensor(states,actions,log_probs,returns,advs,TD_errors,rewards)
        for indicies in self.minibatch(N):
            states_b = states[indicies]
            actions_b = actions[indicies]
            log_probs_b = log_probs[indicies]
            advs_b = advs[indicies]
            returns_b = returns[indicies]
            TD_errors_b = TD_errors[indicies]
            rewards_b = rewards[indicies]
            
            self.learn(states_b,actions_b,log_probs_b,advs_b,returns_b,TD_errors_b,rewards_b)
        
    def bulk_tensor(self,states,actions,log_probs,returns,advs,TD_errors,rewards):
        states = torch.from_numpy(states).float().to(self.device)
        actions = torch.from_numpy(actions).float().to(self.device)
        log_probs = torch.from_numpy(log_probs).float().to(self.device)
        advs = torch.from_numpy(advs).float().to(self.device)
        TD_errors = torch.from_numpy(TD_errors).float().to(self.device)
        rewards = torch.from_numpy(rewards).float().to(self.device)
        # TO fix negative stride
        returns = torch.flip(torch.from_numpy(np.flip(returns,axis=0).copy()),dims=(0,)).float().to(self.device)
        return states,actions,log_probs,returns,advs,TD_errors,rewards

    def unwrap(self,trajectory):
        # states = torch.from_numpy(np.vstack([e.state for e in trajectory])).float().to(self.device)
        # values = torch.from_numpy(np.vstack([e.value for e in trajectory])).float().to(self.device)
        # log_probs = torch.from_numpy(np.vstack([e.log_prob for e in trajectory])).float().to(self.device)
        # rewards = torch.from_numpy(np.vstack([e.reward for e in trajectory])).float().to(self.device)
        # next_states = torch.from_numpy(np.vstack([e.next_state for e in trajectory])).float().to(self.device)
        
        states = np.vstack([e.state for e in trajectory])
        values = [e.value for e in trajectory]
        actions = np.vstack([e.action for e in trajectory])
        log_probs = np.vstack([e.log_prob for e in trajectory])
        rewards = np.vstack([e.reward for e in trajectory])
        next_states = np.vstack([e.next_state for e in trajectory])
        
        return states,actions,values,log_probs,rewards,next_states

    def gae(self,values,rewards):
        """
        Generalized Advantage Estimate

        1d arrays
        """
        N = rewards.shape[0]
        combined = self.gamma*self.gae_lambda
        vs = values[:-1]
        next_vs = values[1:]
        TD_errors = rewards + next_vs - vs
        advs = np.zeros(rewards.shape)
        for index in reversed(range(len(TD_errors))):
            discounts = combined**np.arange(0,N-index)
            advs[index] = np.sum(TD_errors[index:] * discounts)
        return advs,TD_errors

    def n_step_returns(self,rewards):
        N = rewards.shape[0]
        discounts = self.gamma**np.arange(N)
        discounted_returns = rewards * discounts.reshape(N,1)
        returns = discounted_returns.cumsum()[::-1].reshape(N,1)
        return returns

    def learn(self,states,actions,log_probs,advs,returns,TD_errors,rewards):
        """
        Learn on batches from trajectory
        """
        
        new_actions,new_log_probs,new_values = self.policy(states)
        
        # ratio = (new_actions - actions)**2
        # ratio = new_actions / actions
        ratio = (new_log_probs - log_probs)**2

        # ratio = new_log_probs / log_probs
        clip = torch.clamp(ratio,1-self.epsilon,1+self.epsilon)
        clipped_surrogate = torch.min(clip*advs,ratio*advs)

        self.optimizer.zero_grad()
        actor_loss = clipped_surrogate.mean()
        critic_loss = F.smooth_l1_loss(rewards,new_values)
        loss = (actor_loss + critic_loss)
        loss.backward()
        nn.utils.clip_grad_norm_(self.policy.parameters(),self.gradient_clip)
        self.optimizer.step()

    def act(self,state):
        state = self.tensor(state)
        route,log_prob,value = self.policy(state)
        return route.detach().cpu().numpy(),log_prob.detach().cpu().numpy(),value.detach().cpu().numpy()

    def minibatch(self,N):
        index = np.arange(N)
        for _ in range(self.SGD_epoch):
            indicies = np.random.choice(index,self.batch_size)
            yield indicies

    def tensor(self,state):
        return torch.tensor(state).float().to(self.device)

# Train PPO

# PER - Priority Replay Buffer

In [1078]:
"""
Priority Tree.
3 tiered tree structure containing
Root node (Object. sum of all lower values)
Intermediate Node (Object. Root as parent, sums a given slice of the priority array)
Priority Array (Array of priorities, length buffer_size)

The number of Intermediate nodes is calculated by the buffer_size / batch_size.

I_episode: current episode of training

Index: is calculated by i_episode % buffer_size. This loops the index after exceeding the buffer_size.

Indicies: (List) of memory/priority entries

intermediate_dict: maps index to intermediate node. Since each Intermediate node is responsible 
for a given slice of the priority array, given a particular index, it will return the Intermediate node
'responsible' for that index.

## Functions:

Add:
Calculates the priority of each TD error -> (abs(TD_error)+epsilon)**alpha
Stores the priority in the Priority_array.
Updates the sum_tree with the new priority

Update_Priorities:
Updates the index with the latest priority of that sample. As priorities can change over training
for a particular experience

Sample:
Splits the current priority_array based on the number of entries, by the batch_size.
Returns the indicies of those samples and the priorities.

Propogate:
Propogates the new priority value up through the tree
"""

class PriorityTree(object):
    def __init__(self,buffer_size,batch_size,alpha,epsilon):
        self.alpha = alpha
        self.epsilon = epsilon
        self.buffer_size = buffer_size
        self.batch_size = batch_size
        self.batch_indicies = np.arange(0,self.batch_size)

        self.num_intermediate_nodes = math.ceil(buffer_size / batch_size)
        self.current_intermediate_node = 0
        self.root = Node(None)
        self.intermediate_nodes = [Intermediate(self.root,batch_size*x,batch_size*(x+1)) for x in range(self.num_intermediate_nodes)]
        self.priority_array = np.zeros(buffer_size)
        self.intermediate_dict = {}
        for index,node in enumerate(self.intermediate_nodes):
            for key in range((batch_size*(index+1))-batch_size,batch_size*(index+1)):
                self.intermediate_dict[key] = node
        print('Priority Tree: Batch Size {} Buffer size {} Number of intermediate Nodes {}'.format(batch_size,buffer_size,self.num_intermediate_nodes))
        
    def add(self,TD_error,index):
        priority = (abs(TD_error)+self.epsilon)**self.alpha
        self.priority_array[index] = priority
        # Update sum
        propogate(self.intermediate_dict[index],self.priority_array)
    
    def sample(self,index,limit):
        # Sample one experience uniformly from each slice of the priorities
        # if index >= self.buffer_size:
        #     indicies = [random.sample(list(range(sample*self.num_intermediate_nodes,(sample+1)*self.num_intermediate_nodes)),1)[0] for sample in range(self.batch_size)]
        #     # indicies = np.random.sample(np.arange(sample*self.num_intermediate_nodes,(sample+1)*self.num_intermediate_nodes))
        # else:
        spacing = np.linspace(0,limit-self.batch_size,self.batch_size,dtype=np.int)
        random_indicies = np.random.choice(self.batch_indicies,size=self.batch_size)
        indicies = random_indicies + spacing


        # interval = int(index / self.batch_size)
        # indicies = [random.sample(list(range(sample*interval,(sample+1)*interval)),1)[0] for sample in range(self.batch_size)]
#         print('indicies',indicies)
        priorities = self.priority_array[indicies]
        return priorities,indicies
    
    def update_priorities(self,TD_errors,indicies):
#         print('TD_errors',TD_errors)
#         print('TD_errors shape',TD_errors.shape)
        priorities = (np.abs(TD_errors)+self.epsilon)**self.alpha
#         print('priorities shape',priorities.shape)
#         print('indicies shape',len(indicies))
#         print('self.priority_array shape',self.priority_array.shape)
        self.priority_array[indicies] = priorities
        # Update sum
        nodes = [self.intermediate_dict[index] for index in indicies] 
        intermediate_nodes = set(nodes)
        [propogate(node,self.priority_array) for node in intermediate_nodes]
    
class Node(object):
    def __init__(self,parent):
        self.parent = parent
        self.children = []
        self.value = 0
            
    def add_child(self,child):
        self.children.append(child)
    
    def set_value(self,value):
        self.value = value
    
    def sum_children(self):
        return sum([child.value for child in self.children])
            
    def __len__(self):
        return len(self.children)

class Intermediate(Node):
    def __init__(self,parent,start,end):
        self.parent = parent
        self.start = start
        self.end = end
        self.value = 0
        parent.add_child(self)
    
    def sum_leafs(self,arr):
        return np.sum(arr[self.start:self.end])

def propogate(node,arr):
    if node.parent != None:
        node.value = node.sum_leafs(arr)
        propogate(node.parent,arr)
    else:
        node.value = node.sum_children()

In [1079]:
"""
Priority Buffer HyperParameters
alpha(priority or w) dictates how biased the sampling should be towards the TD error. 0 < a < 1
beta(IS) informs the importance of the sample update

The paper uses a sum tree to calculate the priority sum in O(log n) time. As such, i've implemented my own version
of the sum_tree which i call priority tree.

We're increasing beta(IS) from 0.5 to 1 over time
alpha(priority) we're holding constant at 0.5

Lets reimagine the buffer so that you can add a sample and then when you sample from the replay, 
you THEN calculate the TD error the priorities and importances. Then update the priorities based on the new distance.
"""

class PriorityReplayBuffer(object):
    def __init__(self,buffer_size,batch_size,seed,alpha=0.5,beta=0.5,beta_end=1,beta_duration=1e+5,epsilon=7e-5,device=None):
        
        self.seed = random.seed(seed)
        self.buffer_size = buffer_size
        self.batch_size = batch_size
        self.alpha = alpha
        self.beta = beta
        self.beta_end = beta_end
        self.beta_duration = beta_duration
        self.beta_increment = (beta_end - beta) / beta_duration
        self.max_w = 0
        self.epsilon = epsilon
        self.TD_sum = 0
        self.index = 0
        if device == None:
            self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        else:
            self.device = device

        self.experience = namedtuple('experience',field_names=['state','action','reward','next_state','done'])
        self.sum_tree = PriorityTree(buffer_size,batch_size,alpha,epsilon)
        self.memory = {}
    
    def add(self,state,action,reward,next_state,done,TD_error):
        e = self.experience(state,action,reward,next_state,done)
        # add memory to memory and add corresponding priority to the priority tree
        self.memory[self.index] = e
        self.sum_tree.add(TD_error,self.index)
        self.index = (self.index + 1) % self.buffer_size 

    def sample(self):
        # We times the error by these weights for the updates
        # Super inefficient to sum everytime. We could implement the tree sum structure. 
        # Or we could sum once on the first sample and then keep track of what we add and lose from the buffer.
        # priority^a over the sum of the priorities^a = likelyhood of the given choice
        # Anneal beta
        self.update_beta()
        # Get the samples and indicies
        priorities,indicies = self.sum_tree.sample(self.index,len(self))
        # Normalize with the sum
        norm_priorities = priorities / self.sum_tree.root.value
        samples = [self.memory[index] for index in indicies]
        # Importance weights
        importances = [(priority * self.buffer_size)**-self.beta for priority in norm_priorities]
        self.max_w = max(self.max_w,max(importances))
        # Normalize importance weights
#         print('importances',importances)
#         print('self.max_w',self.max_w)
        norm_importances = [importance / self.max_w for importance in importances]
#         print('norm_importances',norm_importances)

        states, actions, rewards, next_states, dones = zip(*samples)

        states = torch.stack(states).float().to(self.device)
        actions = torch.stack(actions).float().to(self.device)
        rewards = torch.from_numpy(np.vstack(rewards)).float().to(self.device)
        next_states = torch.stack(next_states).float().to(self.device)
        dones = torch.from_numpy(np.vstack(dones)).float().to(self.device)

        # states = torch.from_numpy(np.vstack([e.state for e in samples if e is not None])).float().to(self.device)
        # actions = torch.from_numpy(np.vstack([e.action for e in samples if e is not None])).float().to(self.device)
        # rewards = torch.from_numpy(np.vstack([e.reward for e in samples if e is not None])).float().to(self.device)
        # next_states = torch.from_numpy(np.vstack([e.next_state for e in samples if e is not None])).float().to(self.device)
        # dones = torch.from_numpy(np.vstack([e.done for e in samples if e is not None]).astype(int)).float().to(self.device)
        
        return (states,actions,rewards,next_states,dones),indicies,norm_importances

    # TODO
    def update_priorities(self):
        pass
    
    def update_beta(self):
        self.beta += self.beta_increment
        self.beta = min(self.beta,self.beta_end)
    
    def __len__(self):
        return len(self.memory.keys())

# DDPG

In [1080]:
class OUnoise(object):
    def __init__(self,size,seed,mu=0,theta=0.15,sigma=0.2):
        self.size = size
        self.mu = mu * np.ones(size)
        self.theta = theta
        self.sigma = sigma
        self.seed = random.seed(seed)
        self.reset()

    def reset(self):
        self.state = copy.copy(self.mu)

    def sample(self):
        x = self.state
        dx = self.theta * (self.mu - x) + self.sigma * np.random.randn(len(x))
        self.state = x+dx
        return self.state

In [1081]:
def hidden_init(layer):
    fan_in = layer.weight.data.size()[0]
    lim = 1. / np.sqrt(fan_in)
    return (-lim, lim)

def hard_update(source,target):
    for target_param,param in zip(target.parameters(), source.parameters()):
        target_param.data.copy_(param.data)
    
class Critic(nn.Module):
    def __init__(self,seed,nS,nA,hidden_dims=(256,128)):
        super(Critic,self).__init__()
        self.seed = torch.manual_seed(seed)
        self.action_space = nA
        self.state_space = nS
        self.nS = [1]
        self.nA = reduce((lambda x,y: x*y),nA)
        [self.nS.append(x) for x in nS]
        self.nS = tuple(self.nS)
#         print('conv critic',hidden_dims[0],nA,nS)
        self.input_layer = nn.Conv1d(nS[0],hidden_dims[0],kernel_size=1)
        self.action_input = nn.Conv1d(nA[0],hidden_dims[0],kernel_size=1)
#         self.input_layer = nn.Linear(64,hidden_dims[0])
        self.input_bn = nn.BatchNorm1d(hidden_dims[0])
        self.hidden_layers = nn.ModuleList()
        self.hidden_layers.append(nn.Linear(nS[1]+nA[1],hidden_dims[0]))
        for i in range(0,len(hidden_dims)-1):
            hidden_layer = nn.Linear(hidden_dims[i],hidden_dims[i+1])
            self.hidden_layers.append(hidden_layer)
        # self.fc1 = nn.Linear(hidden_dims[0]+nA,hidden_dims[1])
        # self.fc1_bn = nn.BatchNorm1d(hidden_dims[1])
        self.output_layer = nn.Linear(hidden_dims[-1]*hidden_dims[-2],1)
        self.reset_parameters()
        
    def reset_parameters(self):
        self.input_layer.weight.data.uniform_(*hidden_init(self.input_layer))
        for hidden_layer in self.hidden_layers:
            hidden_layer.weight.data.uniform_(*hidden_init(hidden_layer))
        self.output_layer.weight.data.uniform_(-3e-3,3e-3)
        
    def forward(self,obs,action):
        N = obs.size()[0]
        # With batchnorm
#         obs = obs.view(-1)
#         action = action.view(-1)
        # xs = self.input_bn(F.relu(self.input_layer(state)))
        # x = torch.cat((xs,action),dim=1)
        # x = self.fc1_bn(F.relu(self.fc1(x)))
        # transpose obs and action
        action = F.relu(self.action_input(action))
        xs = F.relu(self.input_layer(obs))
        x = torch.cat((xs, action), dim=-1)
        for hidden_layer in self.hidden_layers:
            x = F.relu(hidden_layer(x))
        return self.output_layer(x.view(N,-1))

class Actor(nn.Module):
    def __init__(self,seed,nS,nA,hidden_dims=(256,128)):
        super(Actor,self).__init__()
        
        self.seed = torch.manual_seed(seed)
        self.action_space = nA
        self.state_space = nS
        self.nS = [1]
        self.nA = reduce((lambda x,y: x*y),nA)
        [self.nS.append(x) for x in nS]
        self.nS = tuple(self.nS)

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.input_layer = nn.Conv1d(nS[0],hidden_dims[0],kernel_size=1)
#         self.input_layer = nn.Linear(64,hidden_dims[0])
        self.fc1 = nn.Linear(nS[0],hidden_dims[0])
        self.fc2 = nn.Linear(hidden_dims[0],hidden_dims[1])
        self.output_layer = nn.Linear(hidden_dims[0]*hidden_dims[1],self.nA)
        self.reset_parameters()
        
    def reset_parameters(self):
        self.input_layer.weight.data.uniform_(*hidden_init(self.input_layer))
        self.fc1.weight.data.uniform_(*hidden_init(self.fc1))
        self.output_layer.weight.data.uniform_(-3e-3,3e-3)
        
    def forward(self,state):
        x = state
        if not isinstance(state,torch.Tensor):
            x = torch.tensor(x,dtype=torch.float32,device = self.device) #device = self.device,
            x = x.unsqueeze(0)
#         x = x.view(-1)
#         x = F.relu(self.conv1(x))
        N = state.size()[0]
        x = F.relu(self.input_layer(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = x.view(N,-1)
        x = self.output_layer(x).view(N,self.action_space[0],self.action_space[-1])
        return F.softmax(x,dim=-1)

In [1082]:
class DDPG():
    def __init__(self, nS, nA, actor, critic, config):
        self.nS = nS
        self.nA = nA
        self.action_low = config.action_low
        self.action_high = config.action_high
        self.seed = config.seed

        self.clip_norm = config.clip_norm
        self.tau = config.tau
        self.gamma = config.gamma
        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        self.L2 = config.L2
        self.SGD_epoch = config.SGD_epoch
        # noise
        self.noise = OUnoise(nA,config.seed)
        self.noise_scale = 1.0
        self.noise_decay = config.noise_decay

        # Priority Replay Buffer
        self.batch_size = config.batch_size
        self.buffer_size = config.buffer_size
        self.alpha = config.ALPHA
        self.beta = self.start_beta = config.START_BETA
        self.end_beta = config.END_BETA

        # actors networks
        self.actor = actor(self.seed,nS, nA).to(self.device)
        self.actor_target = actor(self.seed,nS, nA).to(self.device)

        # Param noise
        # self.param_noise = AdaptiveParamNoise()
        # self.actor_perturbed = actor(self.seed,nS, nA).to(self.device)

        # critic networks
        self.critic = critic(self.seed,nS, nA).to(self.device)
        self.critic_target = critic(self.seed,nS, nA).to(self.device)

        # Copy the weights from local to target
        hard_update(self.critic,self.critic_target)
        hard_update(self.actor,self.actor_target)

        # optimizer
        self.actor_opt = optim.Adam(self.actor.parameters(), lr=1e-4, weight_decay=self.L2)
        self.critic_opt = optim.Adam(self.critic.parameters(), lr=1e-3, weight_decay=self.L2)

        # replay buffer
        self.PER = PriorityReplayBuffer(self.buffer_size, self.batch_size,self.seed,alpha=self.alpha,device=self.device)

        # reset agent for training
        self.reset_episode()
        self.it = 0

    def save_weights(self,path):
        params = {}
        params['actor'] = self.actor.state_dict()
        params['critic'] = self.critic.state_dict()
        torch.save(params, path)

    def load_weights(self,path):
        checkpoint = torch.load(path, map_location=self.device)
        self.actor.load_state_dict(checkpoint['actor'])
        self.actor_target.load_state_dict(checkpoint['actor'])
        self.critic.load_state_dict(checkpoint['critic'])
        self.critic_target.load_state_dict(checkpoint['critic'])

    def reset_episode(self):
        self.noise.reset()

    def ddpg_distance_metric(actions1,actions2):
        """
        Computes distance between actions taken by two different policies
        Expects numpy arrays
        """
        diff = actions1-actions2
        mean_diff = np.mean(np.square(diff),axis=0)
        dist = sqrt(np.mean(mean_diff))
        return dist

    def act(self, state):
        with torch.no_grad():
            action = self.actor(self.tensor(state)).cpu().numpy()
        action += self.noise.sample() * self.noise_scale
        # renorm
        if np.min(action) < 0:
            action += np.absolute(np.min(action)) * 2
        action  = action / np.sum(action)
        self.noise_scale = max(self.noise_scale * self.noise_decay, 0.01)
        self.actor.train()
        return action

    def act_perturbed(self,state):
        with torch.no_grad():
            action = self.actor_perturbed(self.tensor(state)).cpu().numpy()
        return action

    def perturbed_update(self):
        hard_update(self.actor,self.actor_perturbed)
        params = self.actor_perturbed.state_dict()
        for name in params:
            if 'ln' in name:
                pass
            param = params[name]
            random = torch.randn(param.shape).to(self.device)
            param += random * self.param_noise.current_stddev
            

    def evaluate(self,state):
        self.actor.eval()
        with torch.no_grad():
            action = self.actor(self.tensor(state)).cpu().numpy()
        return action

    def step(self, obs, actions, rewards, next_obs, dones):
        # cast as torch tensors
        next_obs = torch.from_numpy(next_obs).float().to(self.device)
        obs = torch.from_numpy(obs).float().to(self.device)
        actions = torch.from_numpy(actions).float().to(self.device)
        # Calc TD error
        next_action = self.actor(next_obs)
        next_value = self.critic_target(next_obs,next_action)
        target = rewards + self.gamma * next_value * dones
        local = self.critic(obs,actions)
        TD_error = (target - local).squeeze(0)
        self.PER.add(obs, actions, rewards, next_obs, dones, TD_error)
        for _ in range(self.SGD_epoch):
            samples,indicies,importances = self.PER.sample()
            self.learn(samples,indicies,importances)

    def add_replay_warmup(self,obs,actions,rewards,next_obs,dones):
        next_obs = torch.from_numpy(next_obs).float().to(self.device)
        obs = torch.from_numpy(obs).float().to(self.device)
        actions = torch.from_numpy(actions).float().to(self.device)
        # Calculate TD_error
        next_action = self.actor(next_obs)
        next_value = self.critic_target(next_obs,next_action)
        target = rewards + self.gamma * next_value * dones
        local = self.critic(obs,actions)
        TD_error = (target - local).squeeze(0)
        self.PER.add(obs,actions,np.max(rewards),next_obs,np.max(dones),TD_error)

    def learn(self,samples,indicies,importances):
        
        states, actions, rewards, next_states, dones = samples
        next_states = next_states.squeeze(1)
        states = states.squeeze(1)
        actions = actions.squeeze(1)
        # print('actions shape',actions.shape)
        # print('next_states shape',next_states.shape)
        with torch.no_grad():
              target_actions = self.actor_target(next_states)
        # print('target_actions shape',target_actions.shape)
        next_values = self.critic_target(next_states,target_actions)
        y_target = rewards + self.gamma * next_values * (1-dones)
        y_current = self.critic(states, actions)
        TD_error = y_current - y_target
        # update critic
        critic_loss = ((torch.tensor(importances).to(self.device)*TD_error)**2).mean()
        self.critic.zero_grad()
        critic_loss.backward()
        # torch.nn.utils.clip_grad_norm_(self.critic.parameters(),self.clip_norm)
        self.critic_opt.step()

        # update actor
        local_actions = self.actor(states)
        actor_loss = -self.critic(states, local_actions).mean()
        self.actor.zero_grad()
        actor_loss.backward()
        torch.nn.utils.clip_grad_norm_(self.actor.parameters(),self.clip_norm)
        self.actor_opt.step()

        # Update PER
        TD_errors = TD_error.squeeze(1).detach().cpu().numpy()
        self.PER.sum_tree.update_priorities(TD_errors,indicies)

        # soft update networks
        self.soft_update()

    def soft_update(self):
        """Soft update of target network
        θ_target = τ*θ_local + (1 - τ)*θ_target
        """
        for target_param, param in zip(self.actor_target.parameters(), self.actor.parameters()):
            target_param.data.copy_(self.tau*param.data+(1-self.tau)*target_param.data)
        for target_param, param in zip(self.critic_target.parameters(), self.critic.parameters()):
            target_param.data.copy_(self.tau*param.data+(1-self.tau)*target_param.data)

    def tensor(self, x):
        return torch.from_numpy(x).float().to(self.device)


# Train DDPG agent on elevators

In [1083]:
"""
Actions shape: (N,E,F) Where N = Batch size, E = num elevators, F = num floors
"""

def into_action(probs):
    probs = probs.squeeze(0)
#     print(probs.shape)
    actions = np.zeros(probs.shape)
    choices = np.arange(probs.shape[-1])
#     print('choices',choices)
    for elevator in range(actions.shape[0]):
        action_mask = np.random.choice(choices,p=probs[elevator,:])
#         print('action_mask',action_mask)
        actions[elevator,action_mask] = 1
#         print('actions',actions)
    return actions

In [1093]:
def seed_replay_buffer(env, agent, min_buffer_size):
    printing = False
    obs = env.reset()
    while len(agent.PER) < min_buffer_size:
        # Random actions between 1 and -1
        probs = agent.act(obs)
#         print('agent probs',actions)
        actions = into_action(probs)
#         print('agent action',actions)
        
        next_obs,rewards,dones = env.step(actions,printing)
        # reshape
        agent.add_replay_warmup(obs,probs,rewards,next_obs,dones)
        # Store experience
        if dones:
            obs = env.reset()
        obs = next_obs
    print('finished replay warm up')

def train_ddpg(env, agent, config):
    printing = False
    episodes,tmax = config.episodes,config.tmax
    tic = time.time()
    means = []
    mins = []
    maxes = []
    stds = []
    mean_steps = []
    steps = []
    scores_window = deque(maxlen=100)
    for e in range(1,episodes):
        agent.reset_episode()
        episode_scores = []
        obs = env.reset()
        for t in range(tmax):
            probs = agent.act(obs)
            actions = into_action(probs)
            next_obs,rewards,dones = env.step(actions,printing)
            # Step agent with reshaped observations
            agent.step(obs, probs, rewards, next_obs, dones)
            # Score tracking
            episode_scores.append(rewards)
            obs = next_obs
            if dones:
#                 print('done',t)
                steps.append(int(t))
                break
            
        scores_window.append(np.sum(episode_scores))
        means.append(np.mean(scores_window))
        mins.append(min(scores_window))
        maxes.append(max(scores_window))
        mean_steps.append(np.mean(steps))
        stds.append(np.std(scores_window))
        if e % 3 == 0:
            toc = time.time()
            r_mean = np.mean(scores_window)
            r_max = max(scores_window)
            r_min = min(scores_window)
            r_std = np.std(scores_window)
#             plot(means,maxes,mins,mean_steps,num_agents=2,name=config.name,game='Tennis')
            print("\rEpisode: {} out of {}, Steps {}, Mean steps {:.2f}, Noise {:.2f}, Rewards: mean {:.2f}, min {:.2f}, max {:.2f}, std {:.2f}, Elapsed {:.2f}".format(e,episodes,np.sum(steps),np.mean(steps),agent.noise_scale,r_mean,r_min,r_max,r_std,(toc-tic)/60))
#         if np.mean(scores_window) > config.winning_condition:
#             print('Env solved!')
            # save scores
#             pickle.dump([means,maxes,mins,mean_steps], open(str(config.name)+'_scores.p', 'wb'))
            # save policy
#             agent.save_weights(config.checkpoint_path)
#             break

In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
import pickle
from scipy.interpolate import make_interp_spline, BSpline

def plot(name,means,stds):
     
    length = len(means)
    means = np.array(means)
    stds = np.array(stds)

    mins = means-stds
    maxes = means+stds

    xline = np.linspace(0,length,length*10)
    xfit = np.arange(length)
    
    spl = make_interp_spline(xfit,means,k=3)
    spl2 = make_interp_spline(xfit,mins,k=3)
    spl3 = make_interp_spline(xfit,maxes,k=3)

    means_smooth = spl(xline)
    mins_smooth = spl2(xline)
    maxes_smooth = spl3(xline)

    _, ax = plt.subplots()

    title = str(name)+" performance on Elevators"
    x_label = "Number of Episodes"
    y_label = "Score"

    ax.plot(xline, means_smooth, lw=1, color= '#539caf', alpha = 1, label= 'mean')
    ax.fill_between(xline,mins_smooth,maxes_smooth,color='orange',alpha = 0.4, label = 'Min/Max')

    ax.set_title(title)
    ax.set_xlabel(x_label)
    ax.set_ylabel(y_label)
    # plt.show()
    plt.savefig(str(name)+'_performance.png',bbox_inches='tight')
    plt.close()

In [1094]:
def main(algo):
    N_floors = 5
    N_elevators = 1
    N_people = 1
    # Load the ENV
    env = Hotel(N_floors,N_elevators,N_people)

    # size of each action
    action_space = env.action_space

    # examine the state space 
    state_space = env.state_space
    print('Size of each action: {}, Size of the state space {}'.format(action_space,state_space))
    
    ddpg_config = Config(algo)

    agent = DDPG(state_space, action_space,Actor,Critic,ddpg_config)
    # Fill buffer with random actions up to min buffer size
    seed_replay_buffer(env, agent, ddpg_config.min_buffer_size)
    # Train agent
    train_ddpg(env, agent, ddpg_config)

In [1095]:
main('ddpg')

Size of each action: (1, 5), Size of the state space (5, 5)
Priority Tree: Batch Size 256 Buffer size 10000 Number of intermediate Nodes 40
finished replay warm up
Episode: 3 out of 100, Steps 110, Mean steps 36.67, Noise 0.95, Rewards: mean 36.67, min 22.00, max 52.00, std 12.26, Elapsed 2.66
